# Generate High-Quality Abstracts and Rewritten Articles using GPT-5

This notebook generates high-quality abstracts and rewritten articles for training and validation datasets using OpenAI's GPT-5 (or other high-quality models).

The notebook will:
1. Generate high-quality abstracts that meet all five evaluation criteria (Faithfulness, Coverage, Clarity, Conciseness, Coherence)
2. Rewrite the article content to:
   - Remove noise and placeholders (e.g., equation placeholders, figure references)
   - Condense the content (focusing on introduction, conclusion, and discussion sections)
   - Clean up the text for better readability

The generated abstracts and rewritten articles will replace the existing content in the training data, providing better quality examples for fine-tuning.


In [1]:
import json
import sys
from pathlib import Path
import os

from utils import (
    generate_abstracts_and_articles_batch,
    load_conversations,
    get_openai_client,
    display_text,
    display_message,
)

# Set OpenAI API key if not already set
if 'OPENAI_API_KEY' not in os.environ:
    print("Warning: OPENAI_API_KEY not set. Please set it before running.")
    print("You can set it with: os.environ['OPENAI_API_KEY'] = 'your-key-here'")
else:
    print("✓ OPENAI_API_KEY found")


✓ OPENAI_API_KEY found


In [3]:
# Configuration
MODEL = "gpt-5"  # Use "gpt-4o", "o1-preview", or "gpt-4-turbo"
TEMPERATURE = 1.0
MAX_WORKERS = 1000  # Number of parallel API calls

# File paths
BASE_DIR = Path("/Users/ryanarman/code/lab/arxiv_abstract/data")
TRAIN_INPUT = BASE_DIR / "arxiv_summarization_train_instruct.jsonl"
VAL_INPUT = BASE_DIR / "arxiv_summarization_val_instruct.jsonl"

# Output paths
TRAIN_OUTPUT = BASE_DIR / "arxiv_summarization_train_instruct_article_gpt5.jsonl"
VAL_OUTPUT = BASE_DIR / "arxiv_summarization_val_instruct_article_gpt5.jsonl"

print(f"Model: {MODEL}")
print(f"Temperature: {TEMPERATURE}")
print(f"Max workers: {MAX_WORKERS}")
print(f"\nTrain input: {TRAIN_INPUT}")
print(f"Train output: {TRAIN_OUTPUT}")
print(f"\nVal input: {VAL_INPUT}")
print(f"Val output: {VAL_OUTPUT}")


Model: gpt-5
Temperature: 1.0
Max workers: 1000

Train input: /Users/ryanarman/code/lab/arxiv_abstract/data/arxiv_summarization_train_instruct.jsonl
Train output: /Users/ryanarman/code/lab/arxiv_abstract/data/arxiv_summarization_train_instruct_article_gpt5.jsonl

Val input: /Users/ryanarman/code/lab/arxiv_abstract/data/arxiv_summarization_val_instruct.jsonl
Val output: /Users/ryanarman/code/lab/arxiv_abstract/data/arxiv_summarization_val_instruct_article_gpt5.jsonl


In [4]:
# Load training conversations
print("Loading training data...")
train_conversations = load_conversations(TRAIN_INPUT)
print(f"Loaded {len(train_conversations)} training conversations")

# Load validation conversations
print("\nLoading validation data...")
val_conversations = load_conversations(VAL_INPUT)
print(f"Loaded {len(val_conversations)} validation conversations")


Loading training data...
Loaded 10000 training conversations

Loading validation data...
Loaded 1000 validation conversations


In [5]:
def prepare_conversations_for_generation(conversations):
    """
    Prepare conversations for abstract and article generation.
    Removes assistant messages and keeps only system + user (paper content).
    Updates system message to include instructions for both abstract and article generation.
    """
    prepared = []
    for conv in conversations:
        # Keep only system and user messages
        filtered = [msg for msg in conv if msg.get('role') in ['system', 'user']]
        if len(filtered) >= 2:  # Need at least system and user
            # Update system message to include article rewriting instructions
            updated_conv = []
            for msg in filtered:
                if msg.get('role') == 'system':
                    # Update system message with combined instructions
                    system_content = """You are an expert academic abstract writer and scientific article editor. Your task has two parts:

PART 1: Create a high-quality abstract for an arXiv paper based on the paper content.

The judge evaluates abstracts based on five dimensions:
1. Faithfulness: The abstract must accurately reflect the paper's content without hallucination
2. Coverage: The abstract must include the essential aspects (main problem, approach, and key results)
3. Clarity: The abstract must be understandable and readable
4. Conciseness: The abstract must be focused and not verbose
5. Coherence: The abstract must be logically structured and flow naturally

PART 2: Rewrite the article content to create a clean, condensed version.

The original article is noisy, contains placeholders for equations and figures, and is often too long. Your task is to:
- Remove all placeholders (e.g., @xcite, @xmath, @xref, [figure X], [equation Y], etc.)
- Clean up noisy text and formatting issues
- Condense the content by focusing on the most important sections: Introduction, Conclusion, and Discussion
- Remove redundant or less critical sections (detailed methodology, extensive background, etc.)
- Maintain the core scientific content and key findings
- Ensure the rewritten article is coherent and readable
- Keep the article length reasonable (typically 30-50% of the original length)

OUTPUT FORMAT:
Your response must be formatted exactly as follows:

ABSTRACT:
[Your generated abstract here]

ARTICLE:
[Your rewritten article content here]

Important: Start with "ABSTRACT:" on its own line, followed by the abstract. Then include "ARTICLE:" on its own line, followed by the rewritten article."""
                    updated_conv.append({
                        'role': 'system',
                        'content': system_content
                    })
                else:
                    updated_conv.append(msg)
            prepared.append(updated_conv)
    return prepared

# Prepare training conversations
train_prepared = prepare_conversations_for_generation(train_conversations)
print(f"Prepared {len(train_prepared)} training conversations for generation")

# Prepare validation conversations
val_prepared = prepare_conversations_for_generation(val_conversations)
print(f"Prepared {len(val_prepared)} validation conversations for generation")


Prepared 10000 training conversations for generation
Prepared 1000 validation conversations for generation


In [6]:
# Display a sample prepared conversation
print("Sample prepared conversation structure:")
print(f"Number of messages: {len(train_prepared[0])}")
for i, msg in enumerate(train_prepared[0]):
    print(f"\nMessage {i+1} - Role: {msg['role']}")
    print(f"Content length: {len(msg['content'])} characters")
    if msg['role'] == 'system':
        print("First 200 chars of system message:")
        print(msg['content'][:200] + "...")

Sample prepared conversation structure:
Number of messages: 2

Message 1 - Role: system
Content length: 1673 characters
First 200 chars of system message:
You are an expert academic abstract writer and scientific article editor. Your task has two parts:

PART 1: Create a high-quality abstract for an arXiv paper based on the paper content.

The judge eva...

Message 2 - Role: user
Content length: 26760 characters


In [7]:
# Display system instruction (first 500 chars)
print("System instruction (first 500 characters):")
print("="*80)
print(train_prepared[0][0]['content'][:500] + "...")
print("="*80)
print(f"\nFull system instruction length: {len(train_prepared[0][0]['content'])} characters")


System instruction (first 500 characters):
You are an expert academic abstract writer and scientific article editor. Your task has two parts:

PART 1: Create a high-quality abstract for an arXiv paper based on the paper content.

The judge evaluates abstracts based on five dimensions:
1. Faithfulness: The abstract must accurately reflect the paper's content without hallucination
2. Coverage: The abstract must include the essential aspects (main problem, approach, and key results)
3. Clarity: The abstract must be understandable and readab...

Full system instruction length: 1673 characters


In [8]:
# Generate abstracts and articles for training set
print("="*80)
print("GENERATING ABSTRACTS AND ARTICLES FOR TRAINING SET")
print("="*80)

train_results, train_errors = generate_abstracts_and_articles_batch(
    train_prepared,
    model=MODEL,
    temperature=TEMPERATURE,
    max_workers=MAX_WORKERS,
    show_progress=True
)

print(f"\nTraining set: {len(train_results)} successful, {len(train_errors)} errors")


GENERATING ABSTRACTS AND ARTICLES FOR TRAINING SET
Generating abstracts and articles for 10000 papers with 1000 workers...
Using model: gpt-5


Generating abstracts and articles: 100%|██████████| 10000/10000 [19:07<00:00,  8.71paper/s, success=1e+4, errors=0]



✓ Completed: 10000 successful, 0 errors

Training set: 10000 successful, 0 errors


In [13]:
# Display sample generated abstract and article
idx, result = train_results[2]
print("="*80)
print("SAMPLE GENERATED ABSTRACT")
print("="*80)
display_text(result['abstract'])

print("\n" + "="*80)
print("SAMPLE GENERATED ARTICLE (first 1000 characters)")
print("="*80)
if result.get('article'):
    display_text(result['article'][:10000] + "..." if len(result['article']) > 10000 else result['article'])
    print(f"\nFull article length: {len(result['article'])} characters")
else:
    print("No article generated (check if model response format was correct)")


SAMPLE GENERATED ABSTRACT
Characters: 1,776 | Words: 237 | Lines: 1




SAMPLE GENERATED ARTICLE (first 1000 characters)
Characters: 7,362 | Words: 979 | Lines: 42




Full article length: 7362 characters


In [ ]:
# Generate abstracts and articles for validation set
print("="*80)
print("GENERATING ABSTRACTS AND ARTICLES FOR VALIDATION SET")
print("="*80)

val_results, val_errors = generate_abstracts_and_articles_batch(
    val_prepared,
    model=MODEL,
    temperature=TEMPERATURE,
    max_workers=MAX_WORKERS,
    show_progress=True
)

print(f"\nValidation set: {len(val_results)} successful, {len(val_errors)} errors")


GENERATING ABSTRACTS AND ARTICLES FOR VALIDATION SET
Generating abstracts and articles for 1000 papers with 1000 workers...
Using model: gpt-5


Generating abstracts and articles:  97%|█████████▋| 972/1000 [02:03<00:15,  1.84paper/s, success=972, errors=0]

In [ ]:
# Display sample generated abstract and article from validation set
idx, result = val_results[0]
print("="*80)
print("SAMPLE GENERATED ABSTRACT (VALIDATION SET)")
print("="*80)
display_text(result['abstract'])

print("\n" + "="*80)
print("SAMPLE GENERATED ARTICLE (VALIDATION SET, first 1000 characters)")
print("="*80)
if result.get('article'):
    display_text(result['article'][:10000] + "..." if len(result['article']) > 10000 else result['article'])
    print(f"\nFull article length: {len(result['article'])} characters")
else:
    print("No article generated (check if model response format was correct)")


Characters: 1,597 | Words: 215 | Lines: 1



In [ ]:
def save_generated_abstracts_and_articles(original_conversations, generation_results, output_path):
    """
    Save generated abstracts and rewritten articles in the same format as training data.
    
    Args:
        original_conversations: Original conversations with system/user messages
        generation_results: List of (index, result_dict) tuples from generate_abstracts_and_articles_batch
        output_path: Path to save the output JSONL file
    """
    # Create mappings of index to generated abstract and article
    abstract_map = {idx: result['abstract'] for idx, result in generation_results}
    article_map = {idx: result.get('article', '') for idx, result in generation_results}
    
    saved_count = 0
    with open(output_path, 'w', encoding='utf-8') as f:
        for idx, original_conv in enumerate(original_conversations):
            if idx in abstract_map:
                # Create new conversation with generated abstract and rewritten article
                new_conv = []
                
                # Process original messages
                for msg in original_conv:
                    if msg.get('role') == 'user':
                        # Replace user message with rewritten article if available
                        if idx in article_map and article_map[idx]:
                            new_conv.append({
                                'role': 'user',
                                'content': f"Paper Content:\n{article_map[idx]}"
                            })
                        else:
                            # Keep original user message if no article was generated
                            new_conv.append(msg)
                    elif msg.get('role') == 'system':
                        # Keep system message (it should already have the updated instructions)
                        new_conv.append(msg)
                    elif msg.get('role') == 'assistant':
                        # Skip original assistant message, we'll add the new one
                        pass
                
                # Add generated abstract as assistant message
                new_conv.append({
                    'role': 'assistant',
                    'content': abstract_map[idx]
                })
                
                # Write to file
                json.dump({'messages': new_conv}, f, ensure_ascii=False)
                f.write('\n')
                saved_count += 1
            else:
                # If generation failed, skip this example
                print(f"Warning: Skipping index {idx} (generation failed)")
    
    return saved_count

# Save training results
print("Saving training results...")
train_saved = save_generated_abstracts_and_articles(train_conversations, train_results, TRAIN_OUTPUT)
print(f"Saved {train_saved} training examples to {TRAIN_OUTPUT}")

# Save validation results
print("\nSaving validation results...")
val_saved = save_generated_abstracts_and_articles(val_conversations, val_results, VAL_OUTPUT)
print(f"Saved {val_saved} validation examples to {VAL_OUTPUT}")


Saving training results...
Saved 10000 training examples to /Users/ryanarman/code/lab/arxiv_abstract/data/arxiv_summarization_train_instruct_gpt5.jsonl

Saving validation results...
Saved 1000 validation examples to /Users/ryanarman/code/lab/arxiv_abstract/data/arxiv_summarization_val_instruct_gpt5.jsonl


In [ ]:
# Verify saved results
gen_train_path = "/Users/ryanarman/code/lab/arxiv_abstract/data/arxiv_summarization_train_instruct_article_gpt5.jsonl"
gen_train_conversations = load_conversations(gen_train_path)

print("="*80)
print("VERIFYING SAVED RESULTS")
print("="*80)
print(f"Loaded {len(gen_train_conversations)} conversations from saved file")
print(f"\nFirst conversation has {len(gen_train_conversations[0])} messages")
for i, msg in enumerate(gen_train_conversations[0]):
    print(f"  Message {i+1}: role={msg['role']}, length={len(msg['content'])} chars")

print("\n" + "="*80)
print("SAMPLE GENERATED ABSTRACT (from saved file)")
print("="*80)
display_message(gen_train_conversations[0], "assistant")

print("\n" + "="*80)
print("SAMPLE REWRITTEN ARTICLE (from saved file, first 500 chars)")
print("="*80)
user_msg = None
for msg in gen_train_conversations[0]:
    if msg['role'] == 'user':
        user_msg = msg['content']
        break
if user_msg:
    # Remove "Paper Content:\n" prefix if present
    article_content = user_msg.replace("Paper Content:\n", "", 1)
    display_text(article_content[:500] + "..." if len(article_content) > 500 else article_content)

Role: ASSISTANT
Characters: 1,553 | Words: 210 | Lines: 1



In [ ]:
# Verify saved validation results
gen_val_path = "/Users/ryanarman/code/lab/arxiv_abstract/data/arxiv_summarization_val_instruct_article_gpt5.jsonl"
gen_val_conversations = load_conversations(gen_val_path)

print("="*80)
print("VERIFYING SAVED VALIDATION RESULTS")
print("="*80)
print(f"Loaded {len(gen_val_conversations)} conversations from saved file")

print("\n" + "="*80)
print("SAMPLE GENERATED ABSTRACT (VALIDATION, from saved file)")
print("="*80)
display_message(gen_val_conversations[0], "assistant")

Role: ASSISTANT
Characters: 1,597 | Words: 215 | Lines: 1



In [ ]:
print("="*80)
print("GENERATION SUMMARY")
print("="*80)
print(f"Model used: {MODEL}")
print(f"\nTraining set:")
print(f"  Generated: {len(train_results)} abstracts and articles")
print(f"  Errors: {len(train_errors)}")
print(f"  Saved to: {TRAIN_OUTPUT}")
print(f"\nValidation set:")
print(f"  Generated: {len(val_results)} abstracts and articles")
print(f"  Errors: {len(val_errors)}")
print(f"  Saved to: {VAL_OUTPUT}")

# Show statistics about generated content
if train_results:
    print("\n" + "="*80)
    print("GENERATION STATISTICS")
    print("="*80)
    abstracts_with_articles = sum(1 for _, result in train_results if result.get('article'))
    avg_abstract_len = sum(len(result['abstract']) for _, result in train_results) / len(train_results)
    avg_article_len = sum(len(result.get('article', '')) for _, result in train_results) / len(train_results)
    
    print(f"Abstracts generated: {len(train_results)}")
    print(f"Articles generated: {abstracts_with_articles}")
    print(f"Average abstract length: {avg_abstract_len:.0f} characters")
    print(f"Average article length: {avg_article_len:.0f} characters")
    
    print("\n" + "="*80)
    print("SAMPLE GENERATED ABSTRACT")
    print("="*80)
    idx, result = train_results[0]
    print(f"\nIndex {idx}:")
    print(result['abstract'][:500] + "..." if len(result['abstract']) > 500 else result['abstract'])
    
    if result.get('article'):
        print("\n" + "="*80)
        print("SAMPLE GENERATED ARTICLE (first 500 chars)")
        print("="*80)
        print(result['article'][:500] + "..." if len(result['article']) > 500 else result['article'])


GENERATION SUMMARY
Model used: gpt-5

Training set:
  Generated: 10000 abstracts
  Errors: 0
  Saved to: /Users/ryanarman/code/lab/arxiv_abstract/data/arxiv_summarization_train_instruct_gpt5.jsonl

Validation set:
  Generated: 1000 abstracts
  Errors: 0
  Saved to: /Users/ryanarman/code/lab/arxiv_abstract/data/arxiv_summarization_val_instruct_gpt5.jsonl

SAMPLE GENERATED ABSTRACT

Index 0:
Additive models offer flexibility and interpretability while mitigating the curse of dimensionality. We study support vector machines (SVMs) with additive kernels in reproducing kernel Hilbert spaces under general convex, Lipschitz continuous losses with Tikhonov regularization, addressing the open question of whether additive kernels yield substantially faster learning in high dimensions when the additive assumption holds. Our analysis covers heavy-tailed distributions via a shifted loss and do...
